# Entraînement du Chatbot d'Anglais avec LLaMA 3.2

Ce notebook permet d'entraîner (fine-tuner) un modèle LLaMA 3.2 ou similaire sur le dataset de questions-réponses pour créer un chatbot d'apprentissage de l'anglais.

## 1. Configuration de l'environnement

Commençons par installer les bibliothèques nécessaires et configurer l'accès à Google Drive.

In [ ]:
# Montage de Google Drive pour accéder aux données
from google.colab import drive
drive.mount('/content/drive')

# Définir le chemin vers votre dossier projet sur Drive
PROJECT_DIR = "/content/drive/MyDrive/DEEP"  # Ajustez selon votre structure

In [ ]:
# Installation des dépendances
!pip install transformers datasets accelerate peft trl bitsandbytes torch

## 2. Préparation des données

Nous allons préparer les données pour l'entraînement en les divisant en ensembles d'entraînement et de validation.

In [ ]:
import json
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Charger les données
with open(f"{PROJECT_DIR}/data/Question_Reponse_DATA_converted.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Loaded {len(data)} QA pairs")

# Analyser les données
instruction_lengths = [len(item["instruction"].split()) for item in data]
input_lengths = [len(item["input"].split()) for item in data]
output_lengths = [len(item["output"].split()) for item in data]

print("\nData Statistics:")
print(f"Total samples: {len(data)}")
print(f"Average instruction length: {sum(instruction_lengths) / len(instruction_lengths):.1f} words")
print(f"Average input length: {sum(input_lengths) / len(input_lengths):.1f} words")
print(f"Average output length: {sum(output_lengths) / len(output_lengths):.1f} words")
print(f"Min/Max instruction length: {min(instruction_lengths)}/{max(instruction_lengths)} words")
print(f"Min/Max input length: {min(input_lengths)}/{max(input_lengths)} words")
print(f"Min/Max output length: {min(output_lengths)}/{max(output_lengths)} words")

# Diviser les données
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)
print(f"Split data into {len(train_data)} training samples and {len(val_data)} validation samples")

# Créer le dossier de sortie
os.makedirs(f"{PROJECT_DIR}/data/processed", exist_ok=True)

# Sauvegarder les données
with open(f"{PROJECT_DIR}/data/processed/train.json", 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open(f"{PROJECT_DIR}/data/processed/val.json", 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

print("Data preparation completed!")

## 3. Configuration du modèle et du fine-tuning

Nous allons utiliser LLaMA 3.2 1B Instruct et la technique LoRA pour un fine-tuning efficace.

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

# Définir les paramètres
model_name = "meta-llama/Meta-Llama-3.2-1B-Instruct"  # Vous pouvez changer pour d'autres modèles
output_dir = f"{PROJECT_DIR}/models/english_tutor_llama3_1b"
train_file = f"{PROJECT_DIR}/data/processed/train.json"
val_file = f"{PROJECT_DIR}/data/processed/val.json"

# Si vous n'avez pas accès à LLaMA 3.2, vous pouvez utiliser un modèle ouvert comme:
# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# ou
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Créer le dossier de sortie
os.makedirs(output_dir, exist_ok=True)

### Chargement du modèle et du tokenizer

In [ ]:
# Configuration pour la quantification 4 bits avec BitsAndBytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Charger le modèle quantifié
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Afficher des informations sur le modèle
print(f"Model: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

### Chargement des données et configuration de l'entraînement

In [ ]:
# Charger les données d'entraînement et de validation
train_dataset = load_dataset("json", data_files=train_file)["train"]
val_dataset = load_dataset("json", data_files=val_file)["train"]

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# Exemple d'un échantillon
print("\nExample of a training sample:")
print(train_dataset[0])

# Créer la configuration LoRA
peft_config = LoraConfig(
    r=8,                     # Dimension des matrices de rang faible
    lora_alpha=16,           # Paramètre alpha pour LoRA
    target_modules=["q_proj", "v_proj"],  # Modules à adapter
    lora_dropout=0.05,       # Dropout pour régularisation
    bias="none",             # Ne pas adapter les biais
    task_type="CAUSAL_LM"    # Type de tâche
)

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,                   # Nombre d'époques
    per_device_train_batch_size=4,        # Taille de lot
    gradient_accumulation_steps=4,        # Accumulation de gradients
    optim="paged_adamw_32bit",           # Optimiseur
    save_steps=50,                        # Fréquence de sauvegarde
    logging_steps=10,                     # Fréquence de logging
    learning_rate=2e-5,                   # Taux d'apprentissage
    weight_decay=0.001,                   # Régularisation
    fp16=True,                            # Utiliser FP16
    bf16=False,                           # Ne pas utiliser BF16
    max_grad_norm=0.3,                    # Clipping de gradient
    max_steps=-1,                         # Utiliser num_train_epochs
    warmup_ratio=0.03,                    # Ratio de warmup
    group_by_length=True,                 # Grouper par longueur
    lr_scheduler_type="cosine",           # Scheduler
    evaluation_strategy="steps",          # Évaluer par étapes
    eval_steps=50,                        # Fréquence d'évaluation
    report_to="tensorboard",              # Reporter à Tensorboard
    push_to_hub=False,                    # Ne pas pousser sur Hugging Face Hub
)

### Préparation du format de prompt

In [ ]:
# Formatter les prompts pour le modèle
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    prompts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        if input_text.strip():
            prompt = f"[INST] {instruction}\n\nContext: {input_text} [/INST] {output}"
        else:
            prompt = f"[INST] {instruction} [/INST] {output}"
        prompts.append(prompt)
        
    return {"text": prompts}

# Afficher un exemple formaté
sample = formatting_prompts_func({"instruction": [train_dataset[0]["instruction"]], 
                                "input": [train_dataset[0]["input"]], 
                                "output": [train_dataset[0]["output"]]})
print("Example of formatted prompt:")
print(sample["text"][0])

### Entraînement du modèle

In [ ]:
# Configurer le trainer SFT
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=512,
    tokenizer=tokenizer,
)

print("Starting training...")
# Entraîner le modèle
trainer.train()

### Sauvegarde du modèle

In [ ]:
print(f"Saving model to {output_dir}")
# Sauvegarder le modèle
trainer.save_model()
print("Model saved successfully!")

## 4. Test du modèle entraîné

In [ ]:
from transformers import pipeline
from peft import PeftModel

# Charger le modèle entraîné
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(model, output_dir)

# Créer un pipeline de génération
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

# Exemples de test
test_examples = [
    "How do I introduce myself in English?",
    "What's the difference between 'been' and 'gone'?",
    "Can you explain the present perfect tense?",
]

print("\nTesting model on examples:")
for example in test_examples:
    prompt = f"[INST] {example} [/INST]"
    print(f"\nPrompt: {example}")
    result = pipe(prompt)[0]["generated_text"]
    
    # Extraire la réponse (après [/INST])
    response = result.split("[/INST]")[-1].strip()
    print(f"Response: {response}")

## 5. Exportation du modèle pour l'application Streamlit

In [ ]:
# Compresser le modèle pour faciliter le téléchargement
!cd {output_dir} && tar -czvf ../english_tutor_model.tar.gz .

print(f"Model compressed and saved to {PROJECT_DIR}/models/english_tutor_model.tar.gz")
print("You can download this file and extract it to use in your Streamlit app.")